In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import random
from API.FileReader import readPkl, readCsv, readGeneratedRegex
from API.FileProcessor import merge_dataframes_to_dict, get_sorted_filtered_sids, prompt_generator, get_grouped_sid_to_regex_dict
from API.regexEvaluation import positive_evaluation, negative_evaluation
from API.Drawer import draw_outcome_histogram, draw_outcome_bar
from API.Drawer import draw_positive_contribution_of_generated_regex, draw_negative_contribution_of_generated_regex

### Post-Processor

In [2]:
def convert_newlines(generated_regex_list):
    for i, regex_text in enumerate(generated_regex_list):
        if isinstance(regex_text, str):  # Check if the element is a string
            generated_regex_list[i] = regex_text.replace("\\r\\n", "\r\n")
    return generated_regex_list

def cleanNewline(text):
    text = text.replace("\\r\\n", "").replace("\r\n", "")
    return text

def cleanAllNewlineinDict(sid_to_unique_texts_dict):
    for sid in sid_to_unique_texts_dict:
        sid_to_unique_texts_dict[sid] = [cleanNewline(text) for text in sid_to_unique_texts_dict[sid]]
    return sid_to_unique_texts_dict

def cleanAllNewlineinList(text_list):
    return [cleanNewline(text) for text in text_list]

### 讀取資料

In [3]:

pop_df = readPkl("dataset/pop_report_with_tknscore_new.pkl")
imap_df = readPkl("dataset/imap_report_with_tknscore_new.pkl")
smtp_df = readPkl("dataset/smtp_report_with_tknscore_new.pkl")
sip_df = readPkl("dataset/sip_report_with_tknscore_new.pkl")
label_df = readCsv("dataset/sid_table(packet).csv")

In [4]:

dfs = [pop_df, imap_df, smtp_df, sip_df]
sid_to_unique_texts_dict = merge_dataframes_to_dict(dfs)


In [5]:
# regexPatern = "^POST /ipp HTTP/1\.1\r\nHost: [\d\.]+:110\r\nUser-Agent: Mozilla/5\.0 zgrab/0\.x\r\nContent-Length: \d{3}\r\nAccept: \*/\*\r\nContent-Type: application/ipp\r\nAccept-Encoding: gzip\r\n\r\n\\x02\\x01$"
# text        = "POST /ipp HTTP/1.1\r\nHost: 61.66.51.40:110\r\nUser-Agent: Mozilla/5.0 zgrab/0.x\r\nContent-Length: 144\r\nAccept: */*\r\nContent-Type: application/ipp\r\nAccept-Encoding: gzip\r\n\r\n\x02\x01"
# print(bool(regex.search(regexPatern, text , regex.DOTALL)))
# print(bool(regex.search("^POST /ipp HTTP/1\.1\r\nHost: [\d\.]+:110\r\nUser-Agent: Mozilla/5\.0 zgrab/0\.x\r\nContent-Length: \d{3}\r\nAccept: \*/\*\r\nContent-Type: application/ipp\r\nAccept-Encoding: gzip\r\n\r\n\\x02\\x01$", "POST /ipp HTTP/1.1\r\nHost: 61.66.51.40:110\r\nUser-Agent: Mozilla/5.0 zgrab/0.x\r\nContent-Length: 144\r\nAccept: */*\r\nContent-Type: application/ipp\r\nAccept-Encoding: gzip\r\n\r\n\x02\x01" , regex.DOTALL)))
# regexPatern2 = "^POST /ipp HTTP/1\.1\\r\\nHost: [\d\.]+:110\\r\\nUser-Agent: Mozilla\/5\.0 zgrab\/0\.x\\r\\nContent-Length: \d+\\r\\nAccept: \*\/\*\\r\\nContent-Type: application\/ipp\\r\\nAccept-Encoding: gzip\\r\\n\\r\\n\\\\x02\\\\x01$"

In [6]:
# GeneratedPatternList = [
# "^(?:[A-Za-z0-9+/]{4}){1,}(?:[A-Za-z0-9+/]{2}==|[A-Za-z0-9+/]{3}=)?(?:\r\n)?$",
# "^[A-Za-z0-9+/=\r\n]+$",
# "^DELE \d+\r\n$",
# "^[Ee][Hh][Ll][Oo] [^\s]+\r\n$",
# "^([A-Za-z0-9+/=\r\n]+)$"                                                                                                                                                                                                                                                                                                 
# ]


# NEGATIVE_SAMPLE_NUMBER = 1000
# SID = '3000026'
# with open('test.txt', 'w') as file:
#     positive_evaluation(GeneratedPatternList, SID, sid_to_unique_texts_dict,file)
#     negative_evaluation(GeneratedPatternList, SID, sid_to_unique_texts_dict, NEGATIVE_SAMPLE_NUMBER,file)
# #後處理 "前面加\
# #把首尾的/去掉

### 實驗參數初始化


In [7]:

NUMBER_OF_ONE_REGEX_GROUP = 9
NEGATIVE_SAMPLE_NUMBER = 1000
EXPERIMENT_NAME = "at least 1"
PROCESS_TYPE = "with post-process" #with/without post-process
MODE = "integer" #fraction/integer
TRESHOLD = 1
#escaped_string = original_string.replace('"', '\\"')
if not os.path.exists(f"experiment/{EXPERIMENT_NAME}"):
    os.makedirs(f"experiment/{EXPERIMENT_NAME}")
generated_regex_list: list = readGeneratedRegex(f"experiment/{EXPERIMENT_NAME}/generated regex.csv")
#後處理 \\r\\n -> \r\n
#generated_regex_list = cleanAllNewlineinList(generated_regex_list)
#sid_to_unique_texts_dict = cleanAllNewlineinDict(sid_to_unique_texts_dict)
grouped_sid_to_regex_dict = get_grouped_sid_to_regex_dict(generated_regex_list, 9, sid_to_unique_texts_dict)
positive_sid_to_success_rate_dict = {}
negative_sid_to_seccess_rate_dict = {}
positive_text_to_error_regex_dict = {}
negative_text_to_error_regex_dict = {}

In [8]:
generated_regex_list

['^DELE \\d+\\r\\n$',
 '^DELE [0-9]+\\r\\n$',
 '^DELE [1-9][0-9]*\\r\\n$',
 '^(DELE \\d+\\r\\n)$',
 '^(DELE [1-9]\\d*\\r\\n)$',
 '^(DELE (?:[1-9]\\d{0,2}|1[0-2]\\d{2})\\r\\n)$',
 '^DELE [0-9]{1,3}\\r\\n$',
 '^DELE \\d{1,3}\\r\\n$',
 '^DELE [1-9][0-9]{0,2}\\r\\n$',
 '^(RETR \\d+\\r\\n|TOP \\d+ 0\\r\\n)$',
 '^(RETR [0-9]+\\r\\n|TOP [0-9]+ 0\\r\\n)$',
 '^(RETR \\d{1,3}\\r\\n|TOP \\d{1,3} 0\\r\\n)$',
 '^(TOP|RETR) [0-9]+( 0)?\\r\\n$',
 '^(TOP [0-9]+ 0\\r\\n|RETR [0-9]+\\r\\n)$',
 '^(TOP|RETR) [0-9]{1,3}( 0)?\\r\\n$',
 '^(RETR|TOP) \\d+( 0)?\\r\\n$',
 '^(RETR \\d+\\r\\n|TOP \\d+ 0\\r\\n)$',
 '^TOP \\d+ 0\\r\\n|RETR \\d+\\r\\n$',
 '^GET http://clientapi\\.ipip\\.net/echo\\.php\\?info=\\d{14} HTTP/1\\.1\\\\r\\\\nHost: clientapi\\.ipip\\.net\\\\r\\\\nAccept: \\*/\\*\\\\r\\\\nPragma: no-cache\\\\r\\\\nUser-Agent: Mozilla/5\\.0 \\(Windows NT 10\\.0; WOW64\\)\\\\r\\\\n\\\\r\\\\n$',
 '^GET http://clientapi\\.ipip\\.net/echo\\.php\\?info=\\d{14} HTTP/1\\.1\\\\r\\\\nHost: clientapi\\.ipip\\.net\\\\r

### 開檔案

In [9]:
directory = f'./experiment/{EXPERIMENT_NAME}/graph/{PROCESS_TYPE}'
if not os.path.exists(directory):
    os.makedirs(directory)

### prompt生成

In [10]:


prompt_generator(sid_to_unique_texts_dict, EXPERIMENT_NAME, 3)

In [11]:
# pattern2 = '^GET http://clientapi\.ipip\.net/echo\.php\?info=\d{14} HTTP/1\.1\r\nHost: clientapi\.ipip\.net\r\nAccept: \*/\*\r\nPragma: no-cache\r\nUser-Agent: Mozilla/5\.0 \(Windows NT 10\.0; WOW64\)\r\n\r\n$'
# test_string2 = '''GET http://clientapi.ipip.net/echo.php?info=20200109135019 HTTP/1.1
# Host: clientapi.ipip.net
# Accept: */*
# Pragma: no-cache
# User-Agent: Mozilla/5.0 (Windows NT 10.0; WOW64)
# '''
# pattern = '^GET http://clientapi\.ipip\.net/echo\.php\?info=\d{14} HTTP/1\.1\nHost: clientapi\.ipip\.net\nAccept: \*/\*\nPragma: no-cache\nUser-Agent: Mozilla/5\.0 \(Windows NT 10\.0; WOW64\)\n\n$'
# test_string = '''GET http://clientapi.ipip.net/echo.php?info=20200109135019 HTTP/1.1
# Host: clientapi.ipip.net
# Accept: */*
# Pragma: no-cache
# User-Agent: Mozilla/5.0 (Windows NT 10.0; WOW64)

# '''
# result = bool(regex.search(pattern, test_string, regex.DOTALL))
# print(result)

In [12]:
grouped_sid_to_regex_dict

{'1161912': ['^DELE \\d+\\r\\n$',
  '^DELE [0-9]+\\r\\n$',
  '^DELE [1-9][0-9]*\\r\\n$',
  '^(DELE \\d+\\r\\n)$',
  '^(DELE [1-9]\\d*\\r\\n)$',
  '^(DELE (?:[1-9]\\d{0,2}|1[0-2]\\d{2})\\r\\n)$',
  '^DELE [0-9]{1,3}\\r\\n$',
  '^DELE \\d{1,3}\\r\\n$',
  '^DELE [1-9][0-9]{0,2}\\r\\n$'],
 '1176066': ['^(RETR \\d+\\r\\n|TOP \\d+ 0\\r\\n)$',
  '^(RETR [0-9]+\\r\\n|TOP [0-9]+ 0\\r\\n)$',
  '^(RETR \\d{1,3}\\r\\n|TOP \\d{1,3} 0\\r\\n)$',
  '^(TOP|RETR) [0-9]+( 0)?\\r\\n$',
  '^(TOP [0-9]+ 0\\r\\n|RETR [0-9]+\\r\\n)$',
  '^(TOP|RETR) [0-9]{1,3}( 0)?\\r\\n$',
  '^(RETR|TOP) \\d+( 0)?\\r\\n$',
  '^(RETR \\d+\\r\\n|TOP \\d+ 0\\r\\n)$',
  '^TOP \\d+ 0\\r\\n|RETR \\d+\\r\\n$'],
 '1239754': ['^GET http://clientapi\\.ipip\\.net/echo\\.php\\?info=\\d{14} HTTP/1\\.1\\\\r\\\\nHost: clientapi\\.ipip\\.net\\\\r\\\\nAccept: \\*/\\*\\\\r\\\\nPragma: no-cache\\\\r\\\\nUser-Agent: Mozilla/5\\.0 \\(Windows NT 10\\.0; WOW64\\)\\\\r\\\\n\\\\r\\\\n$',
  '^GET http://clientapi\\.ipip\\.net/echo\\.php\\?info=\\d{14

In [13]:
directory = f'./experiment/{EXPERIMENT_NAME}/data/{PROCESS_TYPE}'
if not os.path.exists(directory):
    os.makedirs(directory)
for sid, regex_list in grouped_sid_to_regex_dict.items():   
    with open(f'{directory}/{sid}.txt', 'w') as file:
        positive_evaluation(regex_list, sid, sid_to_unique_texts_dict, file, positive_sid_to_success_rate_dict, positive_text_to_error_regex_dict, TRESHOLD,MODE)
        negative_evaluation(regex_list, sid, sid_to_unique_texts_dict, file, negative_sid_to_seccess_rate_dict, negative_text_to_error_regex_dict, TRESHOLD, MODE)
#後處理 "前面加\
#把首尾的/去掉

In [ ]:
sid_to_unique_texts_dict

In [ ]:
# 建立 sid_to_unique_texts_dict
#sid_to_unique_texts_dict = {sid: set(texts) for sid, texts in grouped_sid_to_regex_dict.items()}
# 新的結果字典
draw_positive_contribution_of_generated_regex(grouped_sid_to_regex_dict, sid_to_unique_texts_dict, positive_text_to_error_regex_dict, EXPERIMENT_NAME)
draw_negative_contribution_of_generated_regex(grouped_sid_to_regex_dict, sid_to_unique_texts_dict, negative_text_to_error_regex_dict, EXPERIMENT_NAME)

In [ ]:
# base_path = os.path.join('./experiment', 'test', 'graph/contribution/positive')
# os.makedirs(base_path, exist_ok=True)

In [ ]:
draw_outcome_bar(positive_sid_to_success_rate_dict,EXPERIMENT_NAME, "Positive Accuracy without Post-Processing")
draw_outcome_histogram(positive_sid_to_success_rate_dict, EXPERIMENT_NAME, "Distribution of Positive Success Rates without Post-Processing")
draw_outcome_bar(negative_sid_to_seccess_rate_dict,EXPERIMENT_NAME, "Negative Accuracy without Post-Processing")
draw_outcome_histogram(negative_sid_to_seccess_rate_dict, EXPERIMENT_NAME, "Distribution of Negative Success Rates without Post-Processing")

In [ ]:
import google.generativeai as genai
load_dotenv()

system_prompt = '''
Please find a regular expression to match all packet payloads.
You need to find the similarities in the sentences and generalize the parts where they differ. 
The regular expression is in PCRE format, please be aware to evaluate the validity of the expression you generated under PCRE regulations. 
There will be examples to help you find the patterns. 
[‘DELE 3\r\n’, ‘DELE 128\r\n’, ‘DELE 74\r\n’, ‘DELE 22\r\n’, ‘DELE 70\r\n’] 
These examples show the attacker is trying to delete someone’s email by POP protocol. 
The index of the desired mail is indicated under the DELE command. 
Thus the best regular expression that matches them will be ‘^(DELE)( )(.*)(\r\n)$’ 

With the given example payloads: 
[‘EHLO BtuCBHdSb51.com\r\n’, ‘EHLO 203.187.87.27\r\n’, ‘EHLO slae02Fo9Ep.com\r\n’, ‘EHLO 210.64.37.51\r\n’, ‘EHLO LLb0RwqdbkikFWo.com\r\n’] 
These examples show the attacker is trying to make sure the SMTP server is up and running. The command EHLO works in both lower case and uppercase, after that follows the SMTP server address. 
Thus the best regular expression to match them will be ‘^([E|e][H|h][L|l][O|o])(.*)(\r\n)$’ 

note: Your output must be a in jason format，no need to write Markdown
'''
user_prompt = '''
['DELE 107\r\n', 'DELE 124\r\n', 'DELE 113\r\n', 'DELE 7\r\n', 'DELE 105\r\n', 'DELE 71\r\n', 'DELE 122\r\n', 'DELE 25\r\n', 'DELE 61\r\n', 'DELE 100\r\n', 'DELE 65\r\n', 'DELE 52\r\n', 'DELE 69\r\n', 'DELE 43\r\n', 'DELE 21\r\n', 'DELE 67\r\n', 'DELE 90\r\n', 'DELE 14\r\n', 'DELE 109\r\n', 'DELE 77\r\n', 'DELE 73\r\n', 'DELE 104\r\n', 'DELE 45\r\n', 'DELE 10\r\n', 'DELE 89\r\n', 'DELE 3\r\n', 'DELE 120\r\n', 'DELE 96\r\n', 'DELE 48\r\n', 'DELE 95\r\n', 'DELE 110\r\n', 'DELE 12\r\n', 'DELE 27\r\n', 'DELE 56\r\n', 'DELE 34\r\n', 'DELE 55\r\n', 'DELE 132\r\n', 'DELE 38\r\n', 'DELE 26\r\n', 'DELE 86\r\n', 'DELE 115\r\n', 'DELE 103\r\n', 'DELE 17\r\n', 'DELE 101\r\n', 'DELE 8\r\n', 'DELE 11\r\n', 'DELE 33\r\n', 'DELE 53\r\n', 'DELE 88\r\n', 'DELE 76\r\n']
Please give 3 possible and different regular expressions to match all of the elements. 
You can give only 1 expression if the 3 expressions you find are too similar. 
Let’s work this out in a step-by-step way to make sure we have the right answer. 
To make the expression not too general, make sure the expressions don’t match these negative examples: [‘CAPA\r\n’, ‘CAPA\r\n’, ‘\x15\x03\x01’, ‘GET / HTTP/1.0\r\n\r\n’, ‘r\n\r\n’] 
You only need to give me the three regular expressions in code format.
'''
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

genai.configure(api_key=GEMINI_API_KEY)

model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction=system_prompt,
)

response = model.generate_content(user_prompt)
print(response.text)

In [ ]:
### Gmni
load_dotenv()
import google.generativeai as genai
system_prompt = "You are a professional Regular Expression maker."

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

model=genai.GenerativeModel(
  model_name="gemini-1.5-flash"
)

### OLD CODE

In [ ]:
#獲取各個SID所對應到的text
def map_sid_to_unique_texts(df):
    sid_to_texts = {}
    for _, row in df.iterrows():
        sid = row['sid']
        text = row['text']
        if pd.notnull(text):  # Ensure text is not NaN
            if sid in sid_to_texts:
                sid_to_texts[sid].add(text)
            else:
                sid_to_texts[sid] = {text}
    return sid_to_texts
sid_to_unique_texts = map_sid_to_unique_texts(pop_df)
#print(sid_to_unique_texts)

In [ ]:
#過濾掉text長度小於5的sid
def filter_sids_by_text_length(sit_to_text_dict):
    filtered_sids = set()
    for key, value in sit_to_text_dict.items():
        if len(value) > 5:  # Convert to string to avoid errors with non-string types
            filtered_sids.add(key)
    return filtered_sids

# Assuming pop_df is your DataFrame
filtered_sids = filter_sids_by_text_length(sid_to_unique_texts)
#print(filtered_sids)
#print(len(filtered_sids))

In [ ]:
def get_pcre_by_sid(sid):
    with open('sid_table(packet).csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['SID'] == str(sid):
                # Remove leading and trailing slashes from the pcre value
                print (row['pcre'])
                return row['pcre'].strip('/')
    return None

# Iterate Throush Positive payload text to see if the ans regex is correct    
def is_ans_correct(sid):
    correct = 0
    total = 0
    errorList = []
    ansPattern = get_pcre_by_sid(str(sid))
    print(f"ansPattern: {ansPattern}")
    sid_to_unique_texts = map_sid_to_unique_texts(pop_df)
    texts = sid_to_unique_texts[sid]
    for text in texts:
        total = total + 1
        correct = correct + bool(regex.search(ansPattern, text,regex.DOTALL))
        if not bool(regex.search(r'/^(GET|HEAD|POST|OPTIONS)( )(\/|\/version|\/api.*|\/jsproxy)( )(HTTP\/1\.(?:0|1))(\r\n)(.*)$/', text,regex.DOTALL)):
            errorList.append(text)

    print("answer test")
    print(f"correct: {correct}, total: {total}")
    print(len(errorList))
    for error in errorList:
        print(f"error: {error}")
    return correct, total
is_ans_correct('1783777')

### ChatGPT API


In [ ]:
load_dotenv()
CHAT_GPT_API_KEY = os.getenv('C_API_KEY')
client = OpenAI(
    api_key=CHAT_GPT_API_KEY
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)


### Gemini API

In [ ]:
import google.generativeai as genai
load_dotenv()

system_prompt = '''
Please find a regular expression to match all packet payloads.
You need to find the similarities in the sentences and generalize the parts where they differ. 
The regular expression is in PCRE format, please be aware to evaluate the validity of the expression you generated under PCRE regulations. 
There will be examples to help you find the patterns. 
[‘DELE 3\r\n’, ‘DELE 128\r\n’, ‘DELE 74\r\n’, ‘DELE 22\r\n’, ‘DELE 70\r\n’] 
These examples show the attacker is trying to delete someone’s email by POP protocol. 
The index of the desired mail is indicated under the DELE command. 
Thus the best regular expression that matches them will be ‘^(DELE)( )(.*)(\r\n)$’ 

With the given example payloads: 
[‘EHLO BtuCBHdSb51.com\r\n’, ‘EHLO 203.187.87.27\r\n’, ‘EHLO slae02Fo9Ep.com\r\n’, ‘EHLO 210.64.37.51\r\n’, ‘EHLO LLb0RwqdbkikFWo.com\r\n’] 
These examples show the attacker is trying to make sure the SMTP server is up and running. The command EHLO works in both lower case and uppercase, after that follows the SMTP server address. 
Thus the best regular expression to match them will be ‘^([E|e][H|h][L|l][O|o])(.*)(\r\n)$’ 

note: Your output must be a in jason format，no need to write Markdown
'''
user_prompt = '''
['DELE 107\r\n', 'DELE 124\r\n', 'DELE 113\r\n', 'DELE 7\r\n', 'DELE 105\r\n', 'DELE 71\r\n', 'DELE 122\r\n', 'DELE 25\r\n', 'DELE 61\r\n', 'DELE 100\r\n', 'DELE 65\r\n', 'DELE 52\r\n', 'DELE 69\r\n', 'DELE 43\r\n', 'DELE 21\r\n', 'DELE 67\r\n', 'DELE 90\r\n', 'DELE 14\r\n', 'DELE 109\r\n', 'DELE 77\r\n', 'DELE 73\r\n', 'DELE 104\r\n', 'DELE 45\r\n', 'DELE 10\r\n', 'DELE 89\r\n', 'DELE 3\r\n', 'DELE 120\r\n', 'DELE 96\r\n', 'DELE 48\r\n', 'DELE 95\r\n', 'DELE 110\r\n', 'DELE 12\r\n', 'DELE 27\r\n', 'DELE 56\r\n', 'DELE 34\r\n', 'DELE 55\r\n', 'DELE 132\r\n', 'DELE 38\r\n', 'DELE 26\r\n', 'DELE 86\r\n', 'DELE 115\r\n', 'DELE 103\r\n', 'DELE 17\r\n', 'DELE 101\r\n', 'DELE 8\r\n', 'DELE 11\r\n', 'DELE 33\r\n', 'DELE 53\r\n', 'DELE 88\r\n', 'DELE 76\r\n']
Please give 3 possible and different regular expressions to match all of the elements. 
You can give only 1 expression if the 3 expressions you find are too similar. 
Let’s work this out in a step-by-step way to make sure we have the right answer. 
To make the expression not too general, make sure the expressions don’t match these negative examples: [‘CAPA\r\n’, ‘CAPA\r\n’, ‘\x15\x03\x01’, ‘GET / HTTP/1.0\r\n\r\n’, ‘r\n\r\n’] 
You only need to give me the three regular expressions in code format.
'''
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

genai.configure(api_key=GEMINI_API_KEY)

model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction=system_prompt,
)

response = model.generate_content(user_prompt)
print(response.text)

In [ ]:
def clean_json_string(s):
    # 使用正则表达式移除字符串前后的 '''json 和 '''
    cleaned_string = regex.sub(r"^'''json|'''$", '', s, flags=regex.MULTILINE).strip()
    return cleaned_string
output_string = clean_json_string(input_string)
output_string


### Evaluation Generated Regex
Randonly take 20 positive and 20 negateve data as input to generated regex and answer regex, comparing there result.

In [ ]:
#隨機選擇除了給定的SID以外的100個text
def select_random_texts(sid_to_unique_texts, given_sid, num_texts=100):
    # Filter out the given SID
    filtered_texts = [texts for sid, texts in sid_to_unique_texts.items() if sid != given_sid]
    
    # Flatten the list of lists to a single list of texts
    all_texts = [text for sublist in filtered_texts for text in sublist]
    
    # Randomly select 100 texts, or all texts if there are fewer than 100
    selected_texts = random.sample(all_texts, min(len(all_texts), num_texts))
    
    return selected_texts

def get_pcre_by_sid(sid):
    with open('sid_table(packet).csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['SID'] == str(sid):
                # Remove leading and trailing slashes from the pcre value
                return row['pcre']
    return None

def match_patterns(targetText, GeneratedPatternList,isPositive=True):
    non_matching_patterns = []  # Step 1: Initialize list for non-matching patterns
    # Check each pattern and add non-matching ones to the list
    for i, pattern in enumerate(GeneratedPatternList):
        if not regex.search(pattern, targetText, regex.DOTALL):
            non_matching_patterns.append(f"Pattern {i+1}: {pattern}")
    ourResult = True if (9-len(non_matching_patterns)) >= 7 else False
    with open('evaluation_result.txt', 'w') as file:
        for pattern in non_matching_patterns:
            file.write(pattern)
    #ansMatch = bool(regex.search(ansPattern, targetText))
    return ourResult == isPositive

def positive_answer_evaluation(threeAnsPattern,sid):
    correct = 0
    total = 0
    errorList = []
    ansPattern = get_pcre_by_sid(str(sid))
    print(f"ansPattern: {ansPattern}")
    sid_to_unique_texts = map_sid_to_unique_texts(pop_df)
    texts = sid_to_unique_texts[sid]
    for text in texts:
        total = total + 1
        correct = correct + bool(regex.search(ansPattern, text, regex.DOTALL))
        if not match_patterns(text, threeAnsPattern, True):
            errorList.append(text)

    print("positive test")
    print(f"correct: {correct}, total: {total}")
    print(f"errorList: {errorList}")
    return correct, total
def negative_answer_evaluation(GeneratedPatternList,sid):
    correct = 0
    total = 0
    errorList = []
    ansPattern = get_pcre_by_sid(str(sid))
    texts = select_random_texts(sid_to_unique_texts, sid)
    for text in texts:
        total = total + 1
        if(not regex.search(ansPattern, text, regex.DOTALL)):
            correct = correct + 1
        if  not match_patterns(text, GeneratedPatternList, False):
            errorList.append(text)
    print("negative test")
    print(f"correct: {correct}, total: {total}")
    for error_text in errorList:
        print(error_text)
    print(f"errorList: {errorList}")
    
def positive_evaluation(threeAnsPattern,sid,file):
    correct = 0
    total = 0
    errorList = []
    ansPattern = get_pcre_by_sid(str(sid))
    print(f"ansPattern: {ansPattern}")
    sid_to_unique_texts = map_sid_to_unique_texts(pop_df)
    texts = sid_to_unique_texts[sid]
    for text in texts:
        total = total + 1
        correct = correct + match_patterns(text, threeAnsPattern, True)
        if not match_patterns(text, threeAnsPattern, True):
            errorList.append(text)

    print("positive test")
    print(f"correct: {correct}, total: {total}")
    file.write('print("positive")\n')
    for error in errorList:
        file.write('print(f"errorList: {errorList}")\n')
    return correct, total

def negative_evaluation(GeneratedPatternList,sid,file):
    correct = 0
    total = 0
    errorList = []
    ansPattern = get_pcre_by_sid(str(sid))
    texts = select_random_texts(sid_to_unique_texts, sid)
    for text in texts:
        total = total + 1
        correct = correct + match_patterns(text, GeneratedPatternList, False)
        if  not match_patterns(text, GeneratedPatternList, False):
            errorList.append(text)
    file.write('print("negative")\n')
    for error in errorList:
        file.write('print(f"errorList: {errorList}")\n')
    print(f"correct: {correct}, total: {total}")
    for error_text in errorList:
        
    
regex1 = r"/^([E|e][H|h][L|l][O|o])( [a-zA-Z0-9.-]+)?(\r\n)$/"
regex2 = r"/^EHLO( [a-zA-Z0-9.-]+)?(\r\n)$/"
regex3 = r"/^([Ee][Hh][Ll][Oo])( [a-zA-Z0-9.-]+)?(\r\n)$/"
regex4 = r"^([E|e][H|h][L|l][O|o] [a-zA-Z0-9.-]+(\.[a-zA-Z]{2,})?\r\n)$"
regex5 = r"^([E|e][H|h][L|l][O|o] [a-zA-Z0-9.-]+(\.[a-zA-Z0-9.-]+)+\r\n|[E|e][H|h][L|l][O|o]\r\n)$"
regex6 = r"^([E|e][H|h][L|l][O|o]( [a-zA-Z0-9.-]+(\.[a-zA-Z0-9.-]+)+)?\r\n)$"
regex7 = r"^(EHLO|ehlo)( [^\r\n]+)?(\r\n)$"
regex8 = r"^([E|e][H|h][L|l][O|o])( [a-zA-Z0-9.-]+)?(\r\n)$"
regex9 = r"^([Ee][Hh][Ll][Oo])( [a-zA-Z0-9.-]+)?(\r\n)$"
GeneratedPatternList = [
    regex1, regex2, regex3, regex4, regex5, regex6, regex7, regex8, regex9
]

sid = '1454621'
positive_evaluation(GeneratedPatternList, sid)
negative_evaluation(GeneratedPatternList, sid)

### Helper Function

In [ ]:
def list_csv_columns():
    with open('sid_table(packet).csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        # Print all column names
        print(reader.fieldnames)

In [ ]:
#print filtered_sids and corresponding text
for sid in filtered_sids:
    print(f"sid: {sid}, text: {sid_to_unique_texts[sid]}")

In [ ]:
protocols = [
    {'name': 'pop_df', 'data': pop_df},
    {'name': 'imap_df', 'data': imap_df},
    {'name': 'smtp_df', 'data': smtp_df},
    {'name': 'sip_df', 'data': sip_df}
]

with open('prompt.txt', 'w') as file:
    pass

with open('prompt.txt', 'a') as file:
    for protocol in protocols:
        protocol_name = protocol['name']
        protocol_data = protocol['data']
        
        sid_to_unique_texts = map_sid_to_unique_texts(protocol_data)
        filtered_sids = filter_sids_by_text_length(sid_to_unique_texts)
        
        # Sort the filtered_sids to ensure the output is ordered by SID
        sorted_filtered_sids = sorted(filtered_sids)
        
        file.write(f"Protocol: {protocol_name}\n")
        
        for sid in sorted_filtered_sids:
            texts = list(sid_to_unique_texts[sid])
            random.shuffle(texts)
            selected_texts = texts[:50]
            file.write(f"sid: {sid}, text: {selected_texts}\n")
            file.write("\n")
            file.write("Please find a regular expression to match all packet payloads.\n" + 
                       "You need to find the similarities in the sentences and generalize the parts where they differ. \n" + 
                        "The regular expression is in PCRE format, please be aware to evaluate the validity of the expression you generated under PCRE regulations. \n" + 
                        "There will be examples to help you find the patterns. \n"  +
                        "[‘DELE 3\\r\\n’, ‘DELE 128\\r\\n’, ‘DELE 74\\r\\n’, ‘DELE 22\\r\\n’, ‘DELE 70\\r\\n’] \n" +
                        "These examples show the attacker is trying to delete someone’s email by POP protocol. \n" +
                        "The index of the desired mail is indicated under the DELE command. \n" +
                        "Thus the best regular expression that matches them will be ‘^(DELE)( )(.*)(\\r\\n)$’ \n" + "\n" +
                        "With the given example payloads: \n" +
                        "[‘EHLO BtuCBHdSb51.com\\r\\n’, ‘EHLO 203.187.87.27\\r\\n’, ‘EHLO slae02Fo9Ep.com\\r\\n’, ‘EHLO 210.64.37.51\\r\\n’, ‘EHLO LLb0RwqdbkikFWo.com\\r\\n’] \n" + 
                        "These examples show the attacker is trying to make sure the SMTP server is up and running. " + 
                        "The command EHLO works in both lower case and uppercase, after that follows the SMTP server address. \n" +
                        "Thus the best regular expression to match them will be ‘^([E|e][H|h][L|l][O|o])(.*)(\\r\\n)$’ \n" + "\n" +
                        "Next, with the given payloads: \n")
            file.write(f"{selected_texts}\n")         
            file.write("Please give 3 possible and different regular expressions to match all of the elements. \n" +
                        "You can give only 1 expression if the 3 expressions you find are too similar. \n" + 
                        "Let’s work this out in a step-by-step way to make sure we have the right answer. \n" + 
                        "To make the expression not too general, make sure the expressions don’t match these negative examples: [‘CAPA\\r\\n’, ‘CAPA\\r\\n’, ‘\\x15\\x03\\x01’, ‘GET / HTTP/1.0\\r\\n\\r\\n’, ‘r\\n\\r\\n’] \n" +
                        "You only need to give me the three regular expressions in code format.\n")
            file.write("\n")
        file.write("\n")

In [ ]:
with open('all_texts.txt', 'a') as file:
    for protocol in protocols:
        protocol_name = protocol['name']
        protocol_data = protocol['data']
        
        sid_to_unique_texts = map_sid_to_unique_texts(protocol_data)
        filtered_sids = filter_sids_by_text_length(sid_to_unique_texts)
        
        file.write(f"Protocol: {protocol_name}\n")
        
        for sid in filtered_sids:
            file.write(f"sid: {sid}, text: {sid_to_unique_texts[sid]}\n")
                            
        file.write("\n")

In [ ]:
#獲取特定資料集的所有SID
def get_unique_sids_list(df):
    unique_sids = df[df['text'].notnull()]['sid'].unique()
    return unique_sids
sidList = get_unique_sids_list(pop_df)
print(sidList)
#print(len(sidList))